In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px

CREATE: df_11

In [ ]:
#### import data ####
m1supply_df = pd.read_csv(r'./data/M1SL.csv')
#https://fred.stlouisfed.org/series/M1SL
gdp_df = pd.read_csv(r'./data/GDP.csv')
#https://fred.stlouisfed.org/series/GDP
fed_funds_rate_df = pd.read_csv(r'./data/FEDFUNDS.csv')
#https://fred.stlouisfed.org/series/FEDFUNDS
fed_bal_df = pd.read_csv(r'./data/RESPPANWW.csv')
#https://fred.stlouisfed.org/series/RESPPANWW
gov_debt_df = pd.read_csv(r'./data/GFDEBTN.csv')
#https://fred.stlouisfed.org/series/GFDEBTN

In [ ]:
#### m1supply_df cleaning ####
# rename header
m1supply_df.columns = ['date', 'm1_supply']
# change data type to datetime
m1supply_df['date'] = pd.to_datetime(m1supply_df['date'])
# remove days from data column. yyyy-mm '-dd'
m1supply_df['date'] = m1supply_df['date'].dt.strftime('%Y-%m')
# set index to date and sort
m1supply_df = m1supply_df.set_index('date', drop = False)
m1supply_df = m1supply_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
m1supply_df = m1supply_df.loc['2003-01': '2022-12']
# change data type to float in cpi columns
m1supply_df[['m1_supply']] = m1supply_df[['m1_supply']].astype(float)
m1supply_df['date'] = pd.to_datetime(m1supply_df['date'])

In [ ]:
# ## postprocessing
# print(m1supply_df.info())
# print(m1supply_df.describe(include='all', datetime_is_numeric=True))
# ## final m1supply_df
# m1supply_df.head()

In [ ]:
#### gdp_df cleaning ####
# rename header
gdp_df.columns = ['date', 'gdp']
# change data type to datetime
gdp_df['date'] = pd.to_datetime(gdp_df['date'])
# remove days from data column. yyyy-mm '-dd'
gdp_df['date'] = gdp_df['date'].dt.strftime('%Y-%m')
# set index to date and sort
gdp_df = gdp_df.set_index('date', drop = False)
gdp_df = gdp_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
gdp_df = gdp_df.loc['2003-01': '2022-12']
# change data type to float in cpi columns
gdp_df[['gdp']] = gdp_df[['gdp']].astype(float)
gdp_df['date'] = pd.to_datetime(gdp_df['date'])

In [ ]:
# ## postprocessing
# print(gdp_df.info())
# print(gdp_df.describe(include='all', datetime_is_numeric=True))
# ## final m1supply_df
# gdp_df.head()

In [ ]:
#### fed_funds_rate_df cleaning ####
# rename header
fed_funds_rate_df.columns = ['date', 'fed_funds_rate']
# change data type to datetime
fed_funds_rate_df['date'] = pd.to_datetime(fed_funds_rate_df['date'])
# remove days from data column. yyyy-mm '-dd'
fed_funds_rate_df['date'] = fed_funds_rate_df['date'].dt.strftime('%Y-%m')
# set index to date and sort
fed_funds_rate_df = fed_funds_rate_df.set_index('date', drop = False)
fed_funds_rate_df = fed_funds_rate_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
fed_funds_rate_df = fed_funds_rate_df.loc['2003-01': '2022-12']
# change data type to float in cpi columns
fed_funds_rate_df[['fed_funds_rate']] = fed_funds_rate_df[['fed_funds_rate']].astype(float)
fed_funds_rate_df['date'] = pd.to_datetime(fed_funds_rate_df['date'])

In [ ]:
# ## postprocessing
# print(fed_funds_rate_df.info())
# print(fed_funds_rate_df.describe(include='all', datetime_is_numeric=True))
# ## final fed_funds_rate_df
# fed_funds_rate_df.head()

In [ ]:
#### fed_bal_df cleaning ####
# rename header
fed_bal_df.columns = ['date', 'fed_bal']
# change data type to datetime
fed_bal_df['date'] = pd.to_datetime(fed_bal_df['date'])
# remove days from data column. yyyy-mm '-dd'
fed_bal_df['date'] = fed_bal_df['date'].dt.strftime('%Y-%m')
# remove duplicate date values (time step = monthly)
fed_bal_df = fed_bal_df.drop_duplicates('date')
# set index to date and sort
fed_bal_df = fed_bal_df.set_index('date', drop = False)
fed_bal_df = fed_bal_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
fed_bal_df = fed_bal_df.loc['2003-01': '2022-12']
# change data type to float in cpi columns
fed_bal_df[['fed_bal']] = fed_bal_df[['fed_bal']].astype(float)
fed_bal_df['date'] = pd.to_datetime(fed_bal_df['date'])

In [ ]:
# ## postprocessing
# print(fed_bal_df.info())
# print(fed_bal_df.describe(include='all', datetime_is_numeric=True))
# ## final fed_bal_df
# fed_bal_df.head()

In [ ]:
#### gov_debt_df cleaning ####
# rename header
gov_debt_df.columns = ['date', 'gov_debt']
# change data type to datetime
gov_debt_df['date'] = pd.to_datetime(gov_debt_df['date'])
# remove days from data column. yyyy-mm '-dd'
gov_debt_df['date'] = gov_debt_df['date'].dt.strftime('%Y-%m')
# remove duplicate date values (time step = monthly)
gov_debt_df = gov_debt_df.drop_duplicates('date')
# set index to date and sort
gov_debt_df = gov_debt_df.set_index('date', drop = False)
gov_debt_df = gov_debt_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
gov_debt_df = gov_debt_df.loc['2003-01': '2022-12']
# change data type to float in cpi columns
gov_debt_df[['gov_debt']] = gov_debt_df[['gov_debt']].astype(float)
gov_debt_df['date'] = pd.to_datetime(gov_debt_df['date'])

In [ ]:
# ## postprocessing
# print(gov_debt_df.info())
# print(gov_debt_df.describe(include='all', datetime_is_numeric=True))
# ## final fed_funds_rate_df
# gov_debt_df.head()

In [ ]:
#remove date column (currently indexed)
m1supply_df = m1supply_df.drop(['date'], axis=1)
gdp_df = gdp_df.drop(['date'], axis=1)
fed_funds_rate_df = fed_funds_rate_df.drop(['date'], axis=1)
fed_bal_df = fed_bal_df.drop(['date'], axis=1)
gov_debt_df = gov_debt_df.drop(['date'], axis=1)

In [ ]:
#merge all data frames to df_11
df_11 = pd.merge(m1supply_df, gdp_df, how='outer', on = 'date').merge(fed_funds_rate_df, how='outer', on = 'date').merge(fed_bal_df, how='outer', on = 'date').merge(gov_debt_df, how='outer', on = 'date')
#linear interpolation for government_debt_usd quarterly data
df_11 = df_11.interpolate(limit=1, limit_direction='both').fillna(method='ffill')
#create velocity_money = gdp / m1_supply
df_11 ['velocity_money'] = df_11['gdp'] / df_11['m1_supply']
df_11

In [ ]:
# save data frame as df_11
from pathlib import Path  
filepath = Path('./data_frames/df_11.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_11.to_csv(filepath)

CREATE: df_12

In [ ]:
#### import data ####
nasdaq100_df = pd.read_csv(r'./data/NASDAQ100.csv')
#https://fred.stlouisfed.org/series/NASDAQ100
btc_price_df = pd.read_csv(r'./data/CBBTCUSD.csv')
#https://fred.stlouisfed.org/series/CBBTCUSD
housing_index_df = pd.read_csv(r'./data/USSTHPI.csv')
#https://fred.stlouisfed.org/series/USSTHPI
commod_index_df = pd.read_csv(r'./data/PALLFNFINDEXM.csv')
#https://fred.stlouisfed.org/series/PALLFNFINDEXM
energy_index_df = pd.read_csv(r'./data/PNRGINDEXM.csv')
#https://fred.stlouisfed.org/series/PNRGINDEXM

In [ ]:
#### nasdaq100_df cleaning ####
# rename header
nasdaq100_df.columns = ['date', 'nasdaq100']
# change data type to datetime
nasdaq100_df['date'] = pd.to_datetime(nasdaq100_df['date'])
# remove days from data column. yyyy-mm '-dd'
nasdaq100_df['date'] = nasdaq100_df['date'].dt.strftime('%Y-%m')
# set index to date and sort
nasdaq100_df = nasdaq100_df.set_index('date', drop = False)
nasdaq100_df = nasdaq100_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
nasdaq100_df = nasdaq100_df.loc['2008-01': '2022-12']
# replace values in this dataset that are str('.') 
nasdaq100_df = nasdaq100_df.replace('.', np.nan)
# change data type to float in cpi columns
nasdaq100_df[['nasdaq100']] = nasdaq100_df[['nasdaq100']].astype(float)
nasdaq100_df['date'] = pd.to_datetime(nasdaq100_df['date'])

In [ ]:
# ## postprocessing
# print(nasdaq100_df.info())
# print(nasdaq100_df.describe(include='all', datetime_is_numeric=True))

# ## final nasdaq100_df
# nasdaq100_df.head()

In [ ]:
#### btc_price_df cleaning ####
# rename header
btc_price_df.columns = ['date', 'btc_price']
# change data type to datetime
btc_price_df['date'] = pd.to_datetime(btc_price_df['date'])
# remove days from data column. yyyy-mm '-dd'
btc_price_df['date'] = btc_price_df['date'].dt.strftime('%Y-%m')
# set index to date and sort
btc_price_df = btc_price_df.set_index('date', drop = False)
btc_price_df = btc_price_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
btc_price_df = btc_price_df.loc['2008-01': '2022-12']
# replace values in this dataset that are str('.') 
btc_price_df = btc_price_df.replace('.', np.nan)
# change data type to float in cpi columns
btc_price_df[['btc_price']] = btc_price_df[['btc_price']].astype(float)
btc_price_df['date'] = pd.to_datetime(btc_price_df['date'])

In [ ]:
# ## postprocessing
# print(btc_price_df.info())
# print(btc_price_df.describe(include='all', datetime_is_numeric=True))

# ## final nasdaq100_df
# btc_price_df.head()

In [ ]:
#### home_prices_df cleaning ####
# rename header
housing_index_df.columns = ['date', 'housing_index']
# change data type to datetime
housing_index_df['date'] = pd.to_datetime(housing_index_df['date'])
# remove days from data column. yyyy-mm '-dd'
housing_index_df['date'] = housing_index_df['date'].dt.strftime('%Y-%m')
# set index to date and sort
housing_index_df = housing_index_df.set_index('date', drop = False)
housing_index_df = housing_index_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
housing_index_df = housing_index_df.loc['2008-01': '2022-12']
# replace values in this dataset that are str('.') 
housing_index_df = housing_index_df.replace('.', np.nan)
# change data type to float in cpi columns
housing_index_df[['housing_index']] = housing_index_df[['housing_index']].astype(float)
housing_index_df['date'] = pd.to_datetime(housing_index_df['date'])

In [ ]:
# ## postprocessing
# print(housing_index_df.info())
# print(housing_index_df.describe(include='all', datetime_is_numeric=True))

# ## final nasdaq100_df
# housing_index_df.head()

In [ ]:
#### commod_index_df cleaning ####
# rename header
commod_index_df.columns = ['date', 'commod_index']
# change data type to datetime
commod_index_df['date'] = pd.to_datetime(commod_index_df['date'])
# remove days from data column. yyyy-mm '-dd'
commod_index_df['date'] = commod_index_df['date'].dt.strftime('%Y-%m')
# set index to date and sort
commod_index_df = commod_index_df.set_index('date', drop = False)
commod_index_df = commod_index_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
commod_index_df = commod_index_df.loc['2008-01': '2022-12']
# replace values in this dataset that are str('.') 
commod_index_df = commod_index_df.replace('.', np.nan)
# change data type to float in cpi columns
commod_index_df[['commod_index']] = commod_index_df[['commod_index']].astype(float)
commod_index_df['date'] = pd.to_datetime(commod_index_df['date'])

In [ ]:
# ## postprocessing
# print(commod_index_df.info())
# print(commod_index_df.describe(include='all', datetime_is_numeric=True))

# ## final nasdaq100_df
# commod_index_df.head()

In [ ]:
#### energy_index_df cleaning ####
# rename header
energy_index_df.columns = ['date', 'energy_index']
# change data type to datetime
energy_index_df['date'] = pd.to_datetime(energy_index_df['date'])
# remove days from data column. yyyy-mm '-dd'
energy_index_df['date'] = energy_index_df['date'].dt.strftime('%Y-%m')
# set index to date and sort
energy_index_df = energy_index_df.set_index('date', drop = False)
energy_index_df = energy_index_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
energy_index_df = energy_index_df.loc['2008-01': '2022-12']
# replace values in this dataset that are str('.') 
energy_index_df = energy_index_df.replace('.', np.nan)
# change data type to float in cpi columns
energy_index_df[['energy_index']] = energy_index_df[['energy_index']].astype(float)
energy_index_df['date'] = pd.to_datetime(energy_index_df['date'])

In [ ]:
# ## postprocessing
# print(energy_index_df.info())
# print(energy_index_df.describe(include='all', datetime_is_numeric=True))

# ## final nasdaq100_df
# energy_index_df.head()

In [ ]:
#drop all other rows after first of month
nasdaq100_df = nasdaq100_df.assign(M=nasdaq100_df["date"].dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)
btc_price_df = btc_price_df.assign(M=btc_price_df["date"].dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)
housing_index_df = housing_index_df.assign(M=housing_index_df["date"].dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)
commod_index_df = commod_index_df.assign(M=commod_index_df["date"].dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)
energy_index_df = energy_index_df.assign(M=energy_index_df["date"].dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)

In [ ]:
#remove date column (currently indexed)
nasdaq100_df = nasdaq100_df.drop(['date'], axis=1)
btc_price_df = btc_price_df.drop(['date'], axis=1)
housing_index_df = housing_index_df.drop(['date'], axis=1)
commod_index_df = commod_index_df.drop(['date'], axis=1)
energy_index_df = energy_index_df.drop(['date'], axis=1)

In [ ]:
#merge all data frames to df_12
df_12 = pd.merge(nasdaq100_df, btc_price_df, how='outer', on = 'date').merge(housing_index_df, how='outer', on = 'date').merge(commod_index_df, how='outer', on = 'date').merge(energy_index_df, how='outer', on = 'date')
#linear interpolation for government_debt_usd quarterly data
df_12 = df_12.interpolate(limit=1, limit_direction='both').fillna(method='ffill')
df_12

In [ ]:
# save data frame as df_12
from pathlib import Path  
filepath = Path('./data_frames/df_12.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_12.to_csv(filepath)

CREATE: df_13

In [ ]:
#### import data ####
cpi_index_df = pd.read_csv(r'./data/CPIAUCSL.csv')
#https://fred.stlouisfed.org/series/CPIAUCSL
unit_labor_df = pd.read_csv(r'./data/ULCNFB.csv')
#https://fred.stlouisfed.org/series/ULCNFB
personal_savings_df = pd.read_csv(r'./data/PSAVE.csv')
#https://fred.stlouisfed.org/series/PSAVE
personal_expenses_df = pd.read_csv(r'./data/PCE.csv')
#https://fred.stlouisfed.org/series/PCE
rent_prim_res_df = pd.read_csv(r'./data/CUUR0000SEHA.csv')
#https://fred.stlouisfed.org/series/CUUR0000SEHA

In [ ]:
#### cpi_index_df cleaning ####

# rename header
cpi_index_df.columns = ['date', 'cpi_index']
# change data type to datetime
cpi_index_df['date'] = pd.to_datetime(cpi_index_df['date'])
# remove days from data column. yyyy-mm '-dd'
cpi_index_df['date'] = cpi_index_df['date'].dt.strftime('%Y-%m')
# set index to date and sort
cpi_index_df = cpi_index_df.set_index('date', drop = False)
cpi_index_df = cpi_index_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
cpi_index_df = cpi_index_df.loc['2008-01': '2022-12']
# change data type to float in cpi columns
cpi_index_df[['cpi_index']] = cpi_index_df[['cpi_index']].astype(float)
cpi_index_df['date'] = pd.to_datetime(cpi_index_df['date'])

In [ ]:
#### unit_labor_df cleaning ####

# rename header
unit_labor_df.columns = ['date', 'unit_labor']
# change data type to datetime
unit_labor_df['date'] = pd.to_datetime(unit_labor_df['date'])
# remove days from data column. yyyy-mm '-dd'
unit_labor_df['date'] = unit_labor_df['date'].dt.strftime('%Y-%m')
# set index to date and sort
unit_labor_df = unit_labor_df.set_index('date', drop = False)
unit_labor_df = unit_labor_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
unit_labor_df = unit_labor_df.loc['2008-01': '2022-12']
# change data type to float in cpi columns
unit_labor_df[['unit_labor']] = unit_labor_df[['unit_labor']].astype(float)
unit_labor_df['date'] = pd.to_datetime(unit_labor_df['date'])

In [ ]:
#### personal_savings_df cleaning ####

# rename header
personal_savings_df.columns = ['date', 'personal_savings']
# change data type to datetime
personal_savings_df['date'] = pd.to_datetime(personal_savings_df['date'])
# remove days from data column. yyyy-mm '-dd'
personal_savings_df['date'] = personal_savings_df['date'].dt.strftime('%Y-%m')
# set index to date and sort
personal_savings_df = personal_savings_df.set_index('date', drop = False)
personal_savings_df = personal_savings_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
personal_savings_df = personal_savings_df.loc['2008-01': '2022-12']
# change data type to float in cpi columns
personal_savings_df[['personal_savings']] = personal_savings_df[['personal_savings']].astype(float)
personal_savings_df['date'] = pd.to_datetime(personal_savings_df['date'])

In [ ]:
#### personal_expenses_df cleaning ####

# rename header
personal_expenses_df.columns = ['date', 'personal_expenses']
# change data type to datetime
personal_expenses_df['date'] = pd.to_datetime(personal_expenses_df['date'])
# remove days from data column. yyyy-mm '-dd'
personal_expenses_df['date'] = personal_expenses_df['date'].dt.strftime('%Y-%m')
# set index to date and sort
personal_expenses_df = personal_expenses_df.set_index('date', drop = False)
personal_expenses_df = personal_expenses_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
personal_expenses_df = personal_expenses_df.loc['2008-01': '2022-12']
# change data type to float in cpi columns
personal_expenses_df[['personal_expenses']] = personal_expenses_df[['personal_expenses']].astype(float)
personal_expenses_df['date'] = pd.to_datetime(personal_expenses_df['date'])

In [ ]:
#### rent_prim_res_df cleaning ####

# rename 'rent_prim_res'
rent_prim_res_df.columns = ['date', 'rent_prim_res']
# change data type to datetime
rent_prim_res_df['date'] = pd.to_datetime(rent_prim_res_df['date'])
# remove days from data column. yyyy-mm '-dd'
rent_prim_res_df['date'] = rent_prim_res_df['date'].dt.strftime('%Y-%m')
# replace values in this dataset that are str('.') 
rent_prim_res_df = rent_prim_res_df.replace('.', np.nan)
# set index to date and sort
rent_prim_res_df = rent_prim_res_df.set_index('date', drop = False)
rent_prim_res_df = rent_prim_res_df.sort_index()
# slice time series data from '2008-01' to '2022-12'
rent_prim_res_df = rent_prim_res_df.loc['2008-01': '2022-12']
# change data type to float in cpi columns
rent_prim_res_df[['rent_prim_res']] = rent_prim_res_df[['rent_prim_res']].astype(float)
rent_prim_res_df['date'] = pd.to_datetime(rent_prim_res_df['date'])

In [ ]:
#remove date column (currently indexed)
cpi_index_df = cpi_index_df.drop(['date'], axis=1) #m
unit_labor_df = unit_labor_df.drop(['date'], axis=1) #q
personal_savings_df = personal_savings_df.drop(['date'], axis=1) #q
personal_expenses_df = personal_expenses_df.drop(['date'], axis=1) #m
rent_prim_res_df = rent_prim_res_df.drop(['date'], axis=1) #m

In [ ]:
#merge all data frames to df_13
df_13 = pd.merge(cpi_index_df, unit_labor_df, how='outer', on = 'date').merge(personal_savings_df, how='outer', on = 'date').merge(personal_expenses_df, how='outer', on = 'date').merge(rent_prim_res_df, how='outer', on = 'date')
#linear interpolation for government_debt_usd quarterly data
df_13 = df_13.interpolate(limit=1, limit_direction='both').fillna(method='ffill')
df_13

In [ ]:
# save data frame as df_13
from pathlib import Path  
filepath = Path('./data_frames/df_13.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_13.to_csv(filepath)

Merge df_11, df_12 and df_13 to df_1

In [ ]:
df_1 = pd.merge(df_11, df_12, how='outer', on = 'date').merge(df_13, how='outer', on = 'date')
df_1

In [ ]:
# save data frame as df_1
from pathlib import Path  
filepath = Path('./data_frames/df_1.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_1.to_csv(filepath)